<h1 align="center">Universidad de Buenos Aires.</h1>
<h4 align="center">Trabajo práctico integrador. 
<br> Bases de Datos para Inteligencia Artificial.</h4>
<p align="right"> Por: Sevann Radhak Triztan. </p>


**-** Parámetros de conexión a la base de datos.  
**-** Funciones de inicialización y conexión.  
**-** Creación de la base de datos, sus tablas, columnas y relaciones.  
**-** Inserción de datos.  
**-** Se eliminarn registros defectuosos (con referencias o llaves inexistentes.)  
**Consultas.**  
**1.** Colores más utilizados en los 90.  
**2.** Colores únicos.  
**3.** Tendencia de piezas por sets a lo largo de los años.  
**4.** Temáticas más populares de los 2000.  
**Observaciones.**  

In [4]:
import psycopg2
from psycopg2 import sql, connect, extensions
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import os
import pandas as pd
import shutil
import csv

Parámetros de conexión a la base de datos.

In [5]:
DB_PARAMS = {
    "user": "postgres",
    "password": "<<password>>",
    "host": "127.0.0.1",
    "port": "5432",
    "database": "postgres"  
}

DB_NAME = "lego_database"

Funciones de inicialización y conexión.

In [6]:
def database_exists(conn, db_name):
    with conn.cursor() as cur:
        cur.execute("SELECT 1 FROM pg_database WHERE datname=%s", (db_name,))
        return cur.fetchone() is not None

In [7]:
def execute_sql_from_file(conn, file_path, **kwargs):
    with open(file_path, 'r') as file:
        sql = file.read().format(**kwargs)
    with conn.cursor() as cur:
        cur.execute(sql)

In [18]:
def create_database(connection_params, db_name):
    try:        
        conn = psycopg2.connect(**connection_params)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        with conn.cursor() as cur:
            cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (db_name,))
            exists = cur.fetchone()
            if exists:
                print(f"Database '{db_name}' already exists.")
            else:
                cur.execute(f"CREATE DATABASE {db_name}")                
                print(f"Database '{db_name}' created successfully.")
            DB_PARAMS["database"] = db_name
        conn.close()

    except Exception as e:
        print(f"Error while creating the database: {e}")

In [68]:
def create_tables(db_params):
    sql_files = [
    'create_colors_table.sql',
    'create_part_categories_table.sql',
    'create_parts_table.sql',
    'create_themes_table.sql',
    'create_sets_table.sql',
    'create_inventories_table.sql',
    'create_inventory_parts_table.sql',
    'create_inventory_sets_table.sql'
]

    try:
        with psycopg2.connect(**db_params) as conn:
            conn.set_session(autocommit=True)
            
            for sql_file in sql_files:
                execute_sql_from_file(conn, f'sql/{sql_file}')
                print(f"Table from '{sql_file}' created.")
    except Exception as e:
        print(f"Error creating tables: {e}")

Creación de la base de datos, sus tablas, columnas y relaciones.

In [19]:
create_database(DB_PARAMS, DB_NAME)

Database 'lego_database' already exists.


In [69]:
create_tables(DB_PARAMS)

Table from 'create_colors_table.sql' created.
Table from 'create_part_categories_table.sql' created.
Table from 'create_parts_table.sql' created.
Table from 'create_themes_table.sql' created.
Table from 'create_sets_table.sql' created.
Table from 'create_inventories_table.sql' created.
Table from 'create_inventory_parts_table.sql' created.
Table from 'create_inventory_sets_table.sql' created.


Inserción de datos.

In [97]:
def insert_data_from_csv_to_db(db_params, file_path, table_name):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        columns = next(reader)  
        records = [[None if item == '' else item for item in row] for row in reader] 
        
        conn = connect(**db_params)
        conn.set_isolation_level(extensions.ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = conn.cursor()
        
        placeholders = ', '.join(['%s'] * len(columns))  
        all_placeholders = ', '.join([f"({placeholders})"] * len(records))  
        query = sql.SQL("INSERT INTO {table} ({fields}) VALUES {values} ON CONFLICT DO NOTHING").format(
            table=sql.Identifier(table_name),
            fields=sql.SQL(', ').join(map(sql.Identifier, columns)),
            values=sql.SQL(all_placeholders)
        )

        flat_records = [item for sublist in records for item in sublist]  

        try:
            cursor.execute(query, flat_records)
            print("All records inserted successfully.")
        except psycopg2.IntegrityError as e:
            if "violates foreign key constraint" in str(e):
                missing_part_num = str(e).split("Key (part_num)=(")[1].split(")")[0]
                print(f"Error: part_num {missing_part_num} is not present in the 'parts' table.")
            else:
                print(f"Error executing the query: {e}")
        except (Exception, psycopg2.DatabaseError) as error:
            print(f"Error executing the query: {error}")
        finally:
            cursor.close()
            conn.close()

In [52]:
def execute_query(db_params, query, params=None):
    conn = None
    try:
        conn = psycopg2.connect(**db_params)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = conn.cursor()
        
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        
        if query.strip().lower().startswith(("select", "with")):
            records = cursor.fetchall()
            return records
        else:
            conn.commit()
            print("Query executed successfully.")
            return None
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error executing the query: {error}")
        return None
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [57]:
table = 'colors'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [58]:
table = 'part_categories'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [71]:
table = 'themes'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [72]:
table = 'parts'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [73]:
table = 'sets'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [74]:
table = 'inventories'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


Se eliminarn registros defectuosos (con referencias o llaves inexistentes.)

In [71]:
query = "SELECT DISTINCT part_num FROM parts"
unique_part_nums =  execute_query(DB_PARAMS, query)
unique_part_nums = [part_num[0] for part_num in unique_part_nums]

In [111]:
table = 'inventory_parts'
path = f'./raw/{table}.csv'

shutil.copyfile(path, f'./raw/{table}_OLDER.csv')

df_inventory_parts = pd.read_csv(path)
df_to_keep = df_inventory_parts[df_inventory_parts['part_num'].isin(unique_part_nums)]
df_to_keep.to_csv(path, index=False)

print("File updated and _OLDER copy created.")

File updated and _OLDER copy created.


In [108]:
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [109]:
table = 'inventory_sets'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [56]:
def show_result(result, column_names):
    if result:
        df = pd.DataFrame(result, columns=column_names)
        print(df)
    else:
        print("No results were found or an error occurred.")

# Consultas.

## **1.** Colores más utilizados en los 90.  
Identifica cuáles son los 10 colores más frecuentemente usados en los sets de LEGO durante la década de los 90.

In [62]:
query = """
    SELECT c.name AS color_name, SUM(ip.quantity) AS total_quantity
    FROM sets s
    JOIN inventories i ON s.set_num = i.set_num
    JOIN inventory_parts ip ON i.id = ip.inventory_id
    JOIN parts p ON ip.part_num = p.part_num
    JOIN colors c ON ip.color_id = c.id
    WHERE s.year BETWEEN 1990 AND 1999
    GROUP BY c.name
    ORDER BY total_quantity DESC
    LIMIT 10;
"""

result = execute_query(DB_PARAMS, query)
column_names = ['color_name', 'total_quantity']
show_result(result, column_names)

         color_name  total_quantity
0             Black           61341
1        Light Gray           36189
2             White           32899
3               Red           30469
4            Yellow           25469
5              Blue           19499
6             Green            5437
7         Dark Gray            5246
8             Brown            3065
9  Trans-Neon Green            1601


## **2.** Colores únicos.  
Determina la cantidad de colores que son únicos en toda la base de datos.

In [63]:
query = """
    SELECT COUNT(DISTINCT id) AS unique_colors
    FROM colors;
"""

result = execute_query(DB_PARAMS, query)
column_names = ['total']
show_result(result, column_names)

   total
0    135


## **3.** Tendencia de piezas por sets a lo largo de los años.  
Analiza cómo ha evolucionado la cantidad de piezas incluidas en los sets de LEGO a través del tiempo.

In [64]:
query = """
    WITH yearly_avg_parts AS (
        SELECT s.year, AVG(s.num_parts) AS avg_num_parts
        FROM sets s
        GROUP BY s.year
        ORDER BY s.year
    )
    SELECT 
        year,
        avg_num_parts,
        LAG(avg_num_parts, 1) OVER (ORDER BY year) AS prev_year_avg_num_parts,
        avg_num_parts - LAG(avg_num_parts, 1) OVER (ORDER BY year) AS diff_from_prev_year
    FROM yearly_avg_parts;
"""

result = execute_query(DB_PARAMS, query)
column_names = ['year', 'avg_num_parts', 'prev_year_avg_num_parts', 'diff_from_prev_year']    

show_result(result, column_names)

    year         avg_num_parts prev_year_avg_num_parts   diff_from_prev_year
0   1950   10.1428571428571429                    None                  None
1   1953   16.5000000000000000     10.1428571428571429    6.3571428571428571
2   1954   12.3571428571428571     16.5000000000000000   -4.1428571428571429
3   1955   36.8571428571428571     12.3571428571428571   24.5000000000000000
4   1956   18.5000000000000000     36.8571428571428571  -18.3571428571428571
..   ...                   ...                     ...                   ...
61  2013  181.3440134907251265    149.7658536585365854   31.5781598321885411
62  2014  169.7152875175315568    181.3440134907251265  -11.6287259731935697
63  2015  201.6691729323308271    169.7152875175315568   31.9538854147992703
64  2016  253.0771812080536913    201.6691729323308271   51.4080082757228642
65  2017  260.8209459459459459    253.0771812080536913    7.7437647378922546

[66 rows x 4 columns]


## **4.** Temáticas más populares de los 2000.  
Identifica cuáles fueron las temáticas de sets más populares durante la década de los 2000.

In [66]:
query = """
    SELECT 
        t.name AS theme_name,
        COUNT(s.set_num) AS total_sets
    FROM 
        sets s
    JOIN 
        themes t ON s.theme_id = t.id
    WHERE 
        s.year BETWEEN 2000 AND 2009
    GROUP BY 
        t.name
    ORDER BY 
        total_sets DESC
    LIMIT 10;
"""

result = execute_query(DB_PARAMS, query)
column_names = ['theme_name', 'total_sets']    

show_result(result, column_names)

                theme_name  total_sets
0                     City         142
1              Bulk Bricks         124
2                Basic Set         121
3                  Creator         118
4                  Clikits         105
5                  Technic         104
6  Star Wars Episode 4/5/6          88
7                   Soccer          80
8             Supplemental          73
9       Knights Kingdom II          69


## Observaciones.  
- **Manejo de Valores Nulos en themes:**  
Se reemplazaron las cadenas vacías por None en la columna correspondiente a tipo entero en la tabla themes. Esta modificación fue necesaria para asegurar que los inserts se ejecutaran correctamente sin errores de tipo.  

- **Actualización de Registros en inventory_parts:**
Se añadió la cláusula ON CONFLICT (pk_column) DO UPDATE SET en la inserción de inventory_parts para manejar conflictos por claves primarias duplicadas. Esto permite que los registros existentes sean actualizados automáticamente en caso de conflicto.

- **Proceso de Validación de IDs en inventory_parts:**  
Se implementó un proceso para verificar la existencia de los IDs en la tabla parts. Este proceso compara los registros de inventory_parts y elimina aquellos que apuntan a IDs inexistentes en parts. Además, se creó una copia de seguridad del archivo antes de proceder con la eliminación y la inserción de registros.  

-  **Estado de Otras Tablas:**  
No se presentaron problemas adicionales con otras tablas, y los procesos de inserción y actualización se realizaron sin inconvenientes.  